# Cifar10 Class

In [1]:
# coding: UTF-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os 

import numpy as np

In [2]:
class Cifar10Record(object):
    width = 32
    height = 32
    depth = 3
    
    def set_label(self, label_byte):
        self.label = np.frombuffer(label_byte, dtype=np.uint8)
        
    def set_image(self, image_bytes):
        byte_buffer = np.frombuffer(image_bytes, dtype=np.uint8)
        reshaped_array = np.reshape(byte_buffer,
                                   [self.depth, self.height, self.width])
        self.byte_array = np.transpose(reshaped_array, [1, 2, 0])
        self.byte_array = self.byte_array.astype(np.float32)

In [3]:
class Cifar10Reader(object):
    def __init__(self, filename):
        if not os.path.exists(filename):
            print(filename + " is not exist")
            return
        
        self.bytestream = open(filename, mode="rb")
        
    def close(self):
        if not self.bytestream:
            self.bytestream.close()
            
    def read(self, index):
        result = Cifar10Record()
        
        label_bytes = 1
        image_bytes = result.height * result.width * result.depth
        record_bytes = label_bytes + image_bytes
        
        self.bytestream.seek(record_bytes * index, 0)
        
        result.set_label(self.bytestream.read(label_bytes))
        result.set_image(self.bytestream.read(image_bytes))
        
        return result

# Convert CIFAR 10 Image

In [4]:
# coding: UTF-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import numpy as np
import tensorflow as tf
from PIL import Image


FLAGS = tf.app.flags.FLAGS
#tf.app.flags.DEFINE_string("file", "./data/cifar-10-batches-bin/data_batch_1.bin", "")
#tf.app.flags.DEFINE_integer("offset", 0, "")
#tf.app.flags.DEFINE_integer("length", 16, "")
offset = 0
length = 16

basename = os.path.basename("./data/cifar-10-batches-bin/data_batch_1.bin")
path = os.path.dirname("./data/cifar-10-batches-bin/data_batch_1.bin")

reader = Cifar10Reader("./data/cifar-10-batches-bin/data_batch_1.bin")

stop = offset + length

for index in range(offset, stop):
    image = reader.read(index)
    
    print("label: %d" % image.label)
    imageshow = Image.fromarray(image.byte_array.astype(np.uint8))
    
    file_name = "%s-%02d-%d.png" % (basename, index, image.label)
    file = os.path.join(path, file_name)
    with open(file, mode="wb") as out:
        imageshow.save(out, format="png")

reader.close()

label: 6
label: 9
label: 9
label: 4
label: 1
label: 1
label: 2
label: 7
label: 8
label: 3
label: 4
label: 7
label: 7
label: 2
label: 9
label: 9


# Inference

In [5]:
import tensorflow as tf
import os 
import time
import model

EPOCH = 30
data_dir = "./data/cifar-10-batches-bin/"
checkpoint_dir = "./checkpoint_dir/"

In [6]:
filenames = [
    os.path.join(data_dir, 'data_batch_%d.bin' % i) for i in range(1, 6)
]

In [7]:
with tf.device('/gpu:0'):
     train_placeholder = tf.placeholder(tf.float32,
                                      shape=[32, 32, 3], name="input_image")
     image_node = tf.expand_dims(train_placeholder, 0)

     logits = model.inference(image_node)

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.initialize_all_variables())
    
    total_duration = 0
    
    for epoch in range(1, EPOCH):
        start_time = time.time()
        
        for file_index in range(5):
            print("Epoch %d: %s" % (epoch, filenames[file_index]))
            reader = Cifar10Reader(filenames[file_index])
            
            for index in range(10000):
                image = reader.read(index)
                
                logits_value = sess.run([logits],
                                       feed_dict={
                                           train_placeholder: image.byte_array,
                                       })
                if index % 1000 == 0:
                    print("[%d]: %r" %(image.label, logits_value))
                    
                reader.close()
                
            duration = time.time() - start_time
            total_duration += duration
            
            print('epoch %d duration = %d sec' % (epoch, duration))
            
            tf.train.SummaryWriter(checkpoint_dir, sess.graph)
            
        print('Total duration = %d sec' % total_duration)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1: ./data/cifar-10-batches-bin/data_batch_1.bin
[6]: [array([[-0.00699217,  0.01172052, -0.01157834,  0.00869165,  0.01330956,
         0.00492495,  0.01903891, -0.00826489, -0.0068655 , -0.02597469]], dtype=float32)]
[9]: [array([[-0.00912614,  0.01163021, -0.01306129,  0.00974152,  0.01392974,
         0.00436251,  0.01797695, -0.0100047 , -0.0074282 , -0.02633977]], dtype=float32)]
[7]: [array([[-0.01073989,  0.01089748, -0.01282292,  0.00886368,  0.01384023,
         0.00374595,  0.01829774, -0.00858094, -0.00683134, -0.02643224]], dtype=float32)]
[3]: [array([[-0.00997386,  0.01237816, -0.01174688,  0.00993662,  0.01397108,
         0.00557554,  0.01733606, -0.00875834, -0.00772903, -0.02669028]], dtype=float32)]
[5]: [array([[-0.00829392,  0.01130554, -0.01222941,  0.00880565,  0.01381272,
         0.00574741,  0.01981927, -0.01079153, -0.00700656, -0.02818566]], dtype=float32)]
[6]: [array([[-0.00925

KeyboardInterrupt: 